In [1]:
import numpy as np
import torch
from torch import nn, optim
import torchvision
from torchvision import datasets, transforms
from Models.selective_sequential import *
from Loss.triplet_regularized import *
from session import *
from LR_Schedule.cyclical import Cyclical
from LR_Schedule.cos_anneal import CosAnneal
from LR_Schedule.lr_find import lr_find
from callbacks import *
from validation import *
import Datasets.ImageData as ImageData
from Transforms.ImageTransforms import *
import util
from session import LossMeter, EvalModel
from Layers.flatten import Flatten
from torch.utils.tensorboard import SummaryWriter

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/drake/anaconda3/envs/.

In [2]:
%load_ext autoreload
%autoreload 2

torch.cuda.set_device(0); torch.backends.cudnn.benchmark=True;

/home/drake/anaconda3/envs/.torch/lib/python3.7/site-packages/torch/cuda/__init__.py:135: UserWarning: 
    Found GPU0 GeForce GTX 770 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = datasets.MNIST('/media/drake/MX500/Datasets/mnist/train', download=True, train=True, transform=transform)
partial_trainset = torch.utils.data.dataset.Subset(trainset, np.arange(500))

valset = datasets.MNIST('/media/drake/MX500/Datasets/mnist/test', download=True, train=False, transform=transform)
partial_valset = torch.utils.data.dataset.Subset(valset, np.arange(500))

trainloader = torch.utils.data.DataLoader(partial_trainset, batch_size=64, shuffle=True)
valloader = torch.utils.data.DataLoader(partial_valset, batch_size=64, shuffle=False)

In [4]:
class Identity(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__()
    def forward(self, x):
        return x

select = ['act1', 'act2', 'out']

def make_model(dropout=False, batchnorm=False):
    return SelectiveSequential(
    select,
    {'conv64': nn.Conv2d(1, 64, kernel_size=5, padding=2),
     'bn64': nn.BatchNorm2d(num_features=64) if batchnorm else Identity(),
     'act64': nn.ReLU(True),

     'max1': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-64': nn.Dropout(.05) if dropout else Identity(),  
    
     'conv192': nn.Conv2d(64, 192, kernel_size=5, padding=2),
     'bn192': nn.BatchNorm2d(num_features=192) if batchnorm else Identity(),
     'act192': nn.ReLU(True),   

     'max2': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-192': nn.Dropout(.1) if dropout else Identity(),
    
     'conv384': nn.Conv2d(192, 384, kernel_size=3, padding=1),
     'bn384': nn.BatchNorm2d(num_features=384) if batchnorm else Identity(),
     'act384': nn.ReLU(True),
     'drop-384': nn.Dropout(.15) if dropout else Identity(),
     
     'conv256a': nn.Conv2d(384, 256, kernel_size=3, padding=1),
     'bn256a': nn.BatchNorm2d(num_features=256) if batchnorm else Identity(),
     'act256a': nn.ReLU(True),
     'drop-256a': nn.Dropout(.1) if dropout else Identity(),
     
     'conv256b': nn.Conv2d(256, 256, kernel_size=3, padding=1),
     'bn256b': nn.BatchNorm2d(num_features=256) if batchnorm else Identity(),
     'act256b': nn.ReLU(True),

     'max3': nn.MaxPool2d(kernel_size=2, stride=2),
     'drop-256b': nn.Dropout(.1) if dropout else Identity(),
    
     'flatten': Flatten(),

     'fc1': nn.Linear(3 * 3 * 256, 512),
     'act1': nn.ReLU(True),
     'drop-fc1': nn.Dropout(.05) if dropout else Identity(),

     'fc2': nn.Linear(512, 512),
     'act2': nn.ReLU(True),
     'drop-fc2': nn.Dropout(.05) if dropout else Identity(),

     'out': nn.Linear(512, 10)})

In [5]:
criterion = TripletRegularizedMultiMarginLoss(.5, .5, select)
sess = Session(make_model(False, False), criterion, optim.Adam, 1e-4)

In [6]:
# num_epochs = 50
# validator = EmbeddingSpaceValidator(valloader, select, CustomOneHotAccuracy, 
#                                    model_file="./triplet-reg.ckpt.tar")
# lr_scheduler = CosAnneal(len(trainloader)*50, T_mult=1, lr_min=1e-6)
# schedule = TrainingSchedule(trainloader, [lr_scheduler, validator])

In [ ]:
# sess.train(schedule, 50)

In [ ]:
validator.plot()

In [9]:
sess.load("./all-triplet-reg-MNIST-fulldata-2.ckpt.tar")

In [10]:
total_valloader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)
total_validator = EmbeddingSpaceValidator(total_valloader, [], CustomOneHotAccuracy)

total_validator.run(sess)

In [11]:
np.max(total_validator.val_accuracies), "Best accuracy without reg"

(0.9949, 'Best accuracy without reg')

In [13]:
visualization_set = torch.utils.data.dataset.Subset(valset, np.arange(500))
dataloader = torch.utils.data.DataLoader(visualization_set, batch_size=64, shuffle=False)

tensorboard_embeddings(sess.model, ['max1'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_all_tripletreg')

tensorboard_embeddings(sess.model, ['max2'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_all_tripletreg')

tensorboard_embeddings(sess.model, ['max3'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_all_tripletreg')

tensorboard_embeddings(sess.model, ['act1'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_all_tripletreg')

tensorboard_embeddings(sess.model, ['act2'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_all_tripletreg')

tensorboard_embeddings(sess.model, ['out'], 
                       dataloader, 
                       valset.targets[:500], 
                       1.0 - valset.data[:500].reshape(-1, 1, 28, 28) / 255.0, 
                       './mnist_all_tripletreg')